# Try to recover performance on unlearned task adversarially

## First, load model back in

In [ ]:
edge_masks = True
ioi_uniform = 

"masks/ioi_uniform=<tasks.ioi.IOITask.IOITask_Uniform object at 0x7efd4eb420b0>_edge_masks=True_weight_masks_attn=False_weight_masks_mlp=False_train_base_weights=False_localize_acdcpp=False"